READER

In [1]:
import os
import pandas as pd
import regex as re
from datetime import datetime
import numpy as np
import math
from azure.storage.blob import BlobServiceClient

In [2]:
class Reader():
    def __init__(self, file_path):
        self.__file_path = file_path
        self.__files = [ f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path,f)) ]
        self.dataFrames = []

    # Returns a list with all the file names in a specific folder
    def listFile(self):
        return self.__files

    # Returns a list of all the dataframes in the folder from csv files
    def DfList(self):
        if not self.dataFrames:
            for file in self.__files:
                if file.endswith(".csv"):
                    self.dataFrames.append(pd.read_csv(os.path.join(self.__file_path, file)))
                else:
                    raise ValueError("File must be a CSV.")
        return self.dataFrames

    # Returns a specific dataframe by index 
    def getDfByIndex(self, index):
        self.DfList()
        index -= 1
        if index < 0 or index >= len(self.dataFrames):
            raise ValueError("Index out of range.")
        return self.dataFrames[index]
    
    # Returns the amount of files in the folder
    def getLength(self):
        return len(self.__files)
    

WRITER

In [3]:
class Writer():
    def __init__(self, dataFrame, fileName, folder_path):
        self.df = dataFrame
        self.fn = fileName
        self.folder_path = folder_path
        self.writeCsv()

    # Writes a dataframe to a CSV file
    def writeCsv(self):
        if not isinstance(self.df, pd.DataFrame):
            raise ValueError("Dataframe must be a pandas DataFrame")
        
        self.df.to_csv(os.path.join(self.folder_path, self.fn + '.csv'), index=False)
        return print("File saved successfully")
    
    def writeAzureBlobAndCsv(self):
    self.writeCsv()
    
    conn_str = "DefaultEndpointsProtocol=https;AccountName=batchprocessing94;AccountKey=aH9cA5Xwbv+fELTDQPG3BZaM1AvUAK7LQTX5A6PDYMHTw6EqcXlSvzlw5Aqfs7i3XdJSzgQQw3OZ+AStf56OCw==;EndpointSuffix=core.windows.net"
    container_name = "batchprocessingcontainer"
    blob_name = "real_time_data.csv"
    file_path = "output/processed_data.csv"

    blob_service_client = BlobServiceClient.from_connection_string(conn_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data)
        print("File uploaded to Azure Blob Storage successfully")      
    


IndentationError: expected an indented block after function definition on line 16 (1836272169.py, line 17)

In [4]:
reader = Reader('airflow/dags/files/realtime/')
# filenames = ['test1', 'test2']

# for filename, file in zip(filenames, reader.DfList()):
#     Writer(file, filename, 'output')

df = reader.getDfByIndex(1)



VALIDATOR

In [11]:
def validate_name(value):
    if not isinstance(value, str):
        return False

    return True

def validate_platform(value):
    valid_platforms = {
        'Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA', 'PS4', '3DS', 'N64',
        'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT',
        'SCD', 'WS', 'NG', 'TG16', '3DO', 'GG', 'PCFX'
    }

    if not isinstance(value, str):
        return False
    value = value.strip()
    return value in valid_platforms

def validate_yearOfRelease(value):
    if not isinstance(value, float):
        return False

    if value > datetime.now().year:
        return False

    return True

def validate_genre(value):
    if not isinstance(value, str):
        return False

    valid_genres = {
        'Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle', 'Racing',
        'Role-Playing', 'Shooter', 'Simulation', 'Sports', 'Strategy'
    }

    value = value.strip()
    return value in valid_genres

def validate_publisher(value):
    if not isinstance(value, str):
        return False
    
    value_correct = value.strip()
    if not value_correct:
        return False

    return True

def validate_sales(value):
    if not isinstance(value, float):
        return False

    if value < 0 or value > 100:
        return False
    return True

def validate_global_sales(value):
    if not isinstance(value, float):
        return False

    return True

def validate_scores_counts(value):
    if not isinstance(value, float):
        return False

    if value < 0:
        return False
    return True

def validate_User_Score(value):
    if not isinstance(value, str):
        return False
    
    if value == 'tbd':
        return True

    try:
        value = float(value)
    except ValueError:
        return False

    if value < 0 or value > 10:
        return False

    return True

def validate_rating(value):
    if not isinstance(value, str):
        return False

    valid_ratings = {
        'E', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'
    }

    value = value.strip().upper()
    return value in valid_ratings


def validate_row(row):
    errors = []

    validations = {
        'Name': validate_name,
        'Platform': validate_platform,
        'Year_of_Release': validate_yearOfRelease,
        'Genre': validate_genre,
        'Publisher': validate_publisher,
        'NA_Sales': validate_sales,
        'EU_Sales': validate_sales,
        'JP_Sales': validate_sales,
        'Other_Sales': validate_sales,
        'Global_Sales': validate_global_sales,
        'Critic_Score': validate_scores_counts,
        'Critic_Count': validate_scores_counts,
        'User_Score': validate_User_Score,
        'User_Count': validate_scores_counts,
        'Developer': validate_name,
        'Rating': validate_rating,
    }

    for column, validator in validations.items():
        value = row.get(column)
        if pd.isna(value):
            continue 
        if not validator(value):
            errors.append(f"Invalid value in column '{column}': {value}")

    try:
        if not pd.isna(row['Global_Sales']):
            na = row.get('NA_Sales', 0.0) or 0.0
            eu = row.get('EU_Sales', 0.0) or 0.0
            jp = row.get('JP_Sales', 0.0) or 0.0
            other = row.get('Other_Sales', 0.0) or 0.0
            global_sales = row['Global_Sales']

            expected_total = na + eu + jp + other

            if abs(global_sales - expected_total) > 0.03:
                errors.append(
                    f"Global_Sales mismatch: expected ~{expected_total:.2f}, got {global_sales:.2f}"
                )
    except Exception as e:
        errors.append(f"Error validating Global_Sales: {str(e)}")

    return errors



def validate_dataset(df):
    all_errors = {}
    for index, row in df.iterrows():
        errors = validate_row(row)
        if errors:
            all_errors[index] = errors
    return all_errors

validation_errors = validate_dataset(df)


if validation_errors:
    for row_idx, errors in validation_errors.items():
        print(f"Row {row_idx}:")
        for error in errors:
            print(f"  - {error}")
else:
    print("Validation passed: No errors found.")

Validation passed: No errors found.


PROCESSOR

In [ ]:
def process_data(df):
    mandatory_columns = [
        'Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
        'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
    
    # Drop rows with missing values in mandatory columns
    df = df.dropna(subset=mandatory_columns)

    # Remove duplicates based on mandatory columns
    df_cleaned = df.drop_duplicates(subset=mandatory_columns, keep='first')

    # get the best selling region and the ratio for the best selling region
    def get_best_selling_region_and_ratio(row):
        # Define sales per region
        regions = {
            "North-America": row["NA_Sales"],
            "Europe": row["EU_Sales"],
            "Japan": row["JP_Sales"],
            "Other": row["Other_Sales"]
        }
        
        best_region = max(regions, key=regions.get)
        
        best_region_sales = regions[best_region]
        
        if row["Global_Sales"] != 0:
            sales_ratio = best_region_sales / row["Global_Sales"]
        else:
            sales_ratio = 0  
        
        return pd.Series([best_region, sales_ratio], index=["Best_Selling_Region", "Best_Selling_Region_Sales_Ratio"])

    # apply the function to each row to get the best selling region and ratio
    df_cleaned[['Best_Selling_Region', 'Best_Selling_Region_Sales_Ratio']] = df_cleaned.apply(get_best_selling_region_and_ratio, axis=1)

    # Get the contribution of this game to the total sales of its platform
    platform_sales_total = df_cleaned.groupby("Platform")["Global_Sales"].sum()
    df_cleaned["Platform_Sales_Contribution"] = df_cleaned.apply(
        lambda row: row["Global_Sales"] / platform_sales_total[row["Platform"]] if platform_sales_total[row["Platform"]] != 0 else 0,
        axis=1
    )
    return df_cleaned

df_processed = process_data(df)


In [140]:
Writer(df_processed, 'processed_real_data', 'output')

File saved successfully
